In [1]:
import pandas as pd

En primer lugar, antes de realizar las pruebas para obtener diferentes endpoints (funciones) para el sistema de recomendación, cabe aclarar que se utilizará un sistema de puntuación ponderada, en el que se tenga en cuenta el número de reseñas, además de su promedio de calificaciones.

In [5]:
# Definir la función para calcular la puntuación ponderada
def weighted_rating(df, m, C):
    v = df['num_of_reviews']  # Número de reseñas
    R = df['avg_rating']      # Calificación promedio
    # Fórmula para calcular el Weighted Rating
    return (v/(v+m)) * R + (m/(m+v)) * C

In [13]:
# Filtrar restaurantes por categoría y ciudad
def recomendar_restaurantes_por_sabor(df, tipo_comida, ciudad):
    # Filtrar por tipo de comida y ciudad
    df_filtrado = df[(df['category'].str.contains(tipo_comida, case=False)) & (df['city'] == ciudad)].copy()
    
    # Calcular puntuación ponderada para cada restaurante
    m = df_filtrado['num_of_reviews'].quantile(0.90)  # Umbral para el número mínimo de reseñas
    C = df_filtrado['avg_rating'].mean()  # Calificación promedio de todos los restaurantes
    df_filtrado.loc[:, 'puntuacion'] = weighted_rating(df_filtrado, m, C)  # Usar .loc para evitar el warning
    
    # Ordenar por puntuación
    df_top5 = df_filtrado.sort_values('puntuacion', ascending=False).head(5)
    
    return df_top5[['name', 'category', 'avg_rating', 'num_of_reviews', 'address_depurada','city']]


In [10]:
# Ejemplo de uso
df = pd.read_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\locales_google.parquet')  # Cargar la tabla de negocios
top_restaurantes = recomendar_restaurantes_por_sabor(df, 'Pizza', 'San Francisco')
print(top_restaurantes)

                                            name  \
2594792                   Presidio Pizza Company   
1335286                           Monza Pizzeria   
2444761                   Valencia Pizza & Pasta   
2002146  Tricolore Caffè & Pizzeria and Catering   
1361621                          That's Amore SF   

                                                  category  avg_rating  \
2594792                       Pizza restaurant, Restaurant         4.3   
1335286                                   Pizza restaurant         4.6   
2444761  Italian restaurant, Breakfast restaurant, Pizz...         4.3   
2002146  Italian restaurant, Bagel shop, Cafe, Catering...         4.5   
1361621                                   Pizza restaurant         4.8   

         num_of_reviews           city  
2594792             228  San Francisco  
1335286              78  San Francisco  
2444761             178  San Francisco  
2002146              78  San Francisco  
1361621              35  San Francisc

In [11]:
prueba2 = recomendar_restaurantes_por_sabor(df, 'Burger', 'Santa Barbara')
prueba2

,name,category,avg_rating,num_of_reviews,city
2621702,Jack in the Box,"Fast food restaurant, Breakfast restaurant, Ch...",3.7,518,Santa Barbara


In [14]:
prueba3= recomendar_restaurantes_por_sabor(df, 'coffee', 'Los Angeles')
prueba3

,name,category,avg_rating,num_of_reviews,address_depurada,city
2547300,Philz Coffee,Coffee shop,4.6,478,6430 Sunset Blvd,Los Angeles
2387742,Verve Coffee Roasters,Coffee shop,4.5,848,833 S Spring St,Los Angeles
2543080,Tiago Coffee Bar + Kitchen,"Coffee shop, Breakfast restaurant, Brunch rest...",4.5,628,7080 Hollywood Blvd,Los Angeles
2522477,Coffee Del Mundo,"Coffee roasters, Coffee shop",4.8,77,7414 S Vermont Ave,Los Angeles
2491709,Aquarela by SPLA Coffee Co,Coffee shop,4.7,86,714 N Figueroa St,Los Angeles


Estas primeras pruebas me dan la pauta de que puedo hacer diferentes consultas para mis recomendaciones, basadas en los datasets disponibles.

# Creación sistema de recomendación

Realizada la observación anterior, elegimos cuáles serán las posibles consultas a realizarse, y se definen las siguientes:
- Recomendar restaurantes basados en el tipo de comida y ciudad
- Recomendación por Zona: ingreso determinada ciudad y me indicará los mejores locales gastronómicos.
- Recomendar según palabras clave en las reseñas (busca sólo de acuerdo a comercios en que hayan dejado una descripción los usuarios)
- Recomendar locales según reseñas similares



Para el último caso, necesitaremos reducir la muestra a aquellas reseñas que tengan comentarios de texto.

In [15]:
df_reviews = pd.read_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\reviews_google.parquet')
df_reviews.head(3)

,review_id,user_id,name,time,rating,text,gmap_id
0,1,1.089912e+20,Song Ro,2021-06-01,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,2,1.112903e+20,Rafa Robles,2021-09-02,5,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,3,1.126404e+20,David Han,2020-08-03,4,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [19]:
df_reseñas_muestra = df_reviews[df_reviews['text'] != 'Sin Reseña']
df_reseñas_muestra.head(3)


,review_id,user_id,name,time,rating,text,gmap_id
0,1,1.089912e+20,Song Ro,2021-06-01,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,2,1.112903e+20,Rafa Robles,2021-09-02,5,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,3,1.126404e+20,David Han,2020-08-03,4,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [20]:
df_reseñas_muestra.to_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\reseñas_ml.parquet')

Lo extramos en formato parquet para utilizar directamente en el archivo .py

In [21]:
df = pd.read_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\reseñas_ml.parquet')

Comenzamos con la creación de los diferentes endpoints del modelo

In [22]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar el archivo de datos (ajusta la ruta a tu archivo)
df_business = pd.read_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\locales_google.parquet')  # Tabla de negocios
df_reviews = pd.read_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\reseñas_ml.parquet')  # Tabla de reseñas con datos en 'text'

# Mostrar las primeras filas para asegurarnos de que los datos estén cargados correctamente
df_reviews.head()


,review_id,user_id,name,time,rating,text,gmap_id
0,1,1.089912e+20,Song Ro,2021-06-01,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,2,1.112903e+20,Rafa Robles,2021-09-02,5,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,3,1.126404e+20,David Han,2020-08-03,4,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,4,1.174403e+20,Anthony Kim,2019-07-03,5,Short ribs are very delicious.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
25,8,1.035179e+20,andee tea,2021-02-09,5,Love this place fried garlic chicken was very...,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42


Probaremos la función para recomendar restaurantes por sabor y ciudad

In [24]:
#Primero definimos las funciones

def weighted_rating(x, m, C):
    v = x['num_of_reviews']
    R = x['avg_rating']
    return (v / (v + m) * R) + (m / (v + m) * C)

def recomendar_restaurantes_por_sabor(df, tipo_comida, ciudad):
    # Filtrar por tipo de comida y ciudad
    df_filtrado = df[(df['category'].str.contains(tipo_comida, case=False, na=False)) & (df['city'] == ciudad)]
    
    # Verificar si hay registros
    if df_filtrado.empty:
        return f"No se encontraron restaurantes de {tipo_comida} en {ciudad}."
    
    # Calcular los parámetros para la puntuación ponderada
    m = df_filtrado['num_of_reviews'].quantile(0.90)
    C = df_filtrado['avg_rating'].mean()
    
    # Aplicar la puntuación ponderada
    df_filtrado['puntuacion'] = df_filtrado.apply(lambda x: weighted_rating(x, m, C), axis=1)
    
    # Ordenar y devolver el top 5
    df_top5 = df_filtrado.sort_values('puntuacion', ascending=False).head(5)
    return df_top5[['name', 'avg_rating', 'num_of_reviews', 'city', 'category']]

In [25]:
# Probar la función #1
top_restaurantes = recomendar_restaurantes_por_sabor(df_business, 'Pizza', 'San Francisco')
top_restaurantes

C:\Users\GASTON\AppData\Local\Temp\ipykernel_17104\3594798651.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['puntuacion'] = df_filtrado.apply(lambda x: weighted_rating(x, m, C), axis=1)


,name,avg_rating,num_of_reviews,city,category
2594792,Presidio Pizza Company,4.3,228,San Francisco,"Pizza restaurant, Restaurant"
1335286,Monza Pizzeria,4.6,78,San Francisco,Pizza restaurant
2444761,Valencia Pizza & Pasta,4.3,178,San Francisco,"Italian restaurant, Breakfast restaurant, Pizz..."
2002146,Tricolore Caffè & Pizzeria and Catering,4.5,78,San Francisco,"Italian restaurant, Bagel shop, Cafe, Catering..."
1361621,That's Amore SF,4.8,35,San Francisco,Pizza restaurant


Efectivamente, esta función tiene buenos resultados, pero arroja un warning. De todas formas, se mejorará luego de analizar todas las opciones.

*Ahora continamos con la función para buscar restaurantes en una zona*

In [26]:
def recomendar_restaurantes_por_zona(df, ciudad):
    # Filtrar por ciudad
    df_filtrado = df[df['city'] == ciudad]
    
    if df_filtrado.empty:
        return f"No se encontraron restaurantes en {ciudad}."
    
    # Ordenar por rating promedio y devolver el top 5
    df_top5 = df_filtrado.sort_values('avg_rating', ascending=False).head(5)
    return df_top5[['name', 'avg_rating', 'num_of_reviews', 'category']]


In [27]:
# Probar la función
top_zona = recomendar_restaurantes_por_zona(df_business, 'San Francisco')
top_zona

,name,avg_rating,num_of_reviews,category
2498158,Serafina,5.0,48,Restaurant
1956301,Ham and Cheese Deli,5.0,26,Restaurant
1642943,Pho 2000,5.0,18,Vietnamese restaurant
2209316,Ernest,4.9,28,Restaurant
2554745,Paula's Cafe,4.9,35,"Bar & grill, Breakfast restaurant, Brunch rest..."


Busca correctamente las reseñas, pero no tiene los metadatos para ofrecer la información del comercio.

*Continuamos con la búsqueda de palabras clave en las reseñas*

In [28]:
def buscar_palabras_clave(df_reviews, palabras_clave):
    # Unir las palabras clave en una expresión regular
    palabras_regex = '|'.join([f'\\b{palabra}\\b' for palabra in palabras_clave])
    
    # Filtrar las reseñas que contienen las palabras clave
    df_filtrado = df_reviews[df_reviews['text'].str.contains(palabras_regex, case=False, na=False)]
    
    if df_filtrado.empty:
        return "No se encontraron reseñas con las palabras clave."
    
    # Devolver algunas columnas importantes de las reseñas
    return df_filtrado[['review_id', 'name', 'rating', 'text']].head(10)


In [29]:
# Probar la función
palabras_clave = ['amazing', 'good', 'friendly']
resenas_filtradas = buscar_palabras_clave(df_reviews, palabras_clave)
resenas_filtradas


,review_id,name,rating,text
1,2,Rafa Robles,5,Good very good
27,10,Soongil Choe,5,(Translated by Google) There is a different me...
743,15,john b89,5,"It's circle k, cigs,coffee munchies, they got ..."
1379,29,rick belfiore,4,Good hot dog
1751,34,Don Cote,5,Great food. Friendly and efficient service!
1754,36,Maritza Cornejo,5,Good take out food as always
1757,39,Crys P,5,Everything was so good!
2269,42,Dylan Simon,5,Hotei is the best Japanese restaurant in the c...
2272,44,Scott Zawalski,4,Good ramen and tasty Sushi. I wouldn't go here...
2276,46,Hung Pham,3,it's a good place to go on monday's when ebisu...


In [34]:
# Probar la función
palabras_clave = ['good burger']
resenas_filtradas = buscar_palabras_clave(df_reviews, palabras_clave)
resenas_filtradas

,review_id,name,rating,text
79823,1620,Nicole Hsu,2,My bf and I love a good burger. Umai has a pre...
128189,3244,Rick Bolander,3,Pricey but good burger
194384,10672,Bodied_alex,1,Over priced frozen burger patty not the beezne...
245387,16710,Stephanie Hughes,5,My husband and I are new to the area and wante...
245698,16814,Mark Herwig,4,"Pretty good burger place, not expensive"
269039,19431,Phillip Chacon,4,"Good burger, you can tell in the taste that ev..."
269177,19479,Rob Roy,5,A consistently good burger with grass fed beef...
269208,19489,Jovany D.,4,Really good burger. Shakes are great. Pretty p...
269273,19513,Seth Warner,5,Extremely friendly service and good burger
269282,19515,Eric Jay H,4,Pretty damn good burger. A bit too spendy for me


La búsqueda de las palabras es apropiada, pero una vez más, no cuenta con los metadatos para traer información del comercio.

*Ahora generamos la recomendación de restaurantes por reseña:*

In [38]:
def recomendar_restaurantes_por_reseñas(df_reviews, df_business, nombre_restaurante, top_n=5):
    # Buscar el restaurante en la tabla de negocios (df_business)
    df_restaurante = df_business[df_business['name'].str.contains(nombre_restaurante, case=False, na=False)]
    
    if df_restaurante.empty:
        return f"No se encontraron restaurantes con el nombre {nombre_restaurante}."
    
    # Obtener el gmap_id del primer restaurante encontrado (asumiendo que es único)
    gmap_id = df_restaurante.iloc[0]['gmap_id']
    
    # Filtrar las reseñas que tienen texto y corresponden a este restaurante
    df_filtrado = df_reviews[(df_reviews['text'].notnull()) & (df_reviews['gmap_id'] == gmap_id)]
    
    if df_filtrado.empty:
        return f"No se encontraron reseñas para el restaurante {nombre_restaurante}."
    
    # Resetear el índice del DataFrame filtrado para evitar problemas con el índice
    df_filtrado = df_filtrado.reset_index(drop=True)
    
    # Crear la matriz TF-IDF
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_filtrado['text'])
    
    # Calcular la similitud del coseno entre todas las reseñas
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Obtener el índice del restaurante en la matriz
    idx = 0  # Siempre usamos el primer índice ya que reiniciamos el DataFrame
    
    # Verificar que la matriz de similitud tiene el tamaño correcto
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar por similitud y devolver los más similares
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n + 1]
    
    # Obtener los índices de las reseñas recomendadas
    review_indices = [i[0] for i in sim_scores]
    
    return df_filtrado.iloc[review_indices][['name', 'rating', 'text']]

In [39]:

# Probar la función
recomendaciones = recomendar_restaurantes_por_reseñas(df_reviews, df_business, 'Pizza Hut')
recomendaciones

,name,rating,text
3,Les Schlais,4,Good
1,Cody Buchanan,5,There's a guy named Vlad who works there. He's...
2,Saul Zepeda,5,Always fresh employees are great


Esta función es insuficiente, ya que cuando busco el nombre de un comercio, me trae reseñas del mismo comercio.

# Conclusiones: deben reformularse funciones y trabajar con otros datos

De acuerdo a lo visto, debemos hacer un nuevo dataframe base unificando los campos a través del valor 'gmap_id' para poder obtener, a partir de las reseñas, recomendaciones de los restaurantes (y no de las reseñas propiamente dichas)

**Creación de DataFrame unificado**

In [42]:
# Seleccionar solo las columnas necesarias de df_business
df_business_filtrado = df_business[['gmap_id', 'name', 'address', 'category', 'avg_rating', 'num_of_reviews', 'city']]

# Seleccionar solo las columnas necesarias de df_reviews
df_reviews_filtrado = df_reviews[['gmap_id', 'rating', 'text']]

# Hacer el merge utilizando 'gmap_id' como clave
df_unificado = pd.merge(df_reviews_filtrado, df_business_filtrado, on='gmap_id', how='inner')

# Mostrar las primeras filas para verificar el resultado
df_unificado.head()


,gmap_id,rating,text,name,address,category,avg_rating,num_of_reviews,city
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,Love there korean rice cake.,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",Korean restaurant,4.4,18,Los Angeles
1,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,Good very good,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",Korean restaurant,4.4,18,Los Angeles
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,4,They make Korean traditional food very properly.,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",Korean restaurant,4.4,18,Los Angeles
3,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,Short ribs are very delicious.,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",Korean restaurant,4.4,18,Los Angeles
4,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,5,Love this place fried garlic chicken was very...,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",Restaurant,4.5,18,La Mirada


In [ ]:
# Crear una columna combinada 'category_text' con la categoría repetida 3 veces para darle mayor peso
df_unificado['category_text'] = df_unificado['category'] + ' ' + df_unificado['category'] + ' ' + df_unificado['category'] + ' ' + df_unificado['text']


Esta nueva columna será utilizada para una de las funciones, que trabajan buscando coincidencias a través de TF-IDF (Term frequency – Inverse document frequency).

In [92]:
df_unificado.to_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\ml_unificado.parquet')

**Volvemos a formular el primer endpoint del sistema de recomendación**

Recordemos que el siguiente sistema de recomendación utiliza la puntuación ponderada, según cantidad de reseñas y sus calificaciones

In [46]:
def recomendar_restaurantes_por_sabor_unificado(df, tipo_comida, ciudad, top_n=5):
    '''
    Esta función recibe un DataFrame sobre el que trabaja, el tipo de comida y la ciudad que se busca.
    Devuelve una recomendación de los locales con mejor puntuación ponderada que coincidan con esos criterios.
    '''
    # Filtrar por tipo de comida y ciudad
    df_filtrado = df[(df['category'].str.contains(tipo_comida, case=False, na=False)) & 
                     (df['city'].str.contains(ciudad, case=False, na=False))]
    
    if df_filtrado.empty:
        return f"No se encontraron restaurantes de {tipo_comida} en {ciudad}."
    
    # Agrupar por gmap_id para evitar duplicados
    df_agrupado = df_filtrado.groupby('gmap_id').agg({
        'name': 'first',  # Mantener el primer nombre del restaurante
        'address': 'first',  # Mantener la primera dirección del restaurante
        'category': 'first',  # Mantener la primera categoría
        'avg_rating': 'mean',  # Calcular el promedio de las calificaciones
        'num_of_reviews': 'sum',  # Sumar todas las reseñas
        'rating': 'mean'  # Calcular el promedio de las calificaciones de las reseñas
    }).reset_index()

    # Calcular la puntuación ponderada
    m = df_agrupado['num_of_reviews'].quantile(0.90)
    C = df_agrupado['avg_rating'].mean()
    df_agrupado['puntuacion'] = weighted_rating(df_agrupado, m, C)
    
    # Ordenar por puntuación
    df_top5 = df_agrupado.sort_values('puntuacion', ascending=False).head(top_n)
    
    # Seleccionar las columnas relevantes para mostrar
    return df_top5[['name', 'address', 'category', 'avg_rating', 'num_of_reviews', 'puntuacion']]

In [48]:
# Probar la función con el DataFrame unificado
recomendaciones_sabor = recomendar_restaurantes_por_sabor_unificado(df_unificado, 'Pizza', 'San Francisco')
recomendaciones_sabor

,name,address,category,avg_rating,num_of_reviews,puntuacion
2,Presidio Pizza Company,"Presidio Pizza Company, 1862 Divisadero St, Sa...","Pizza restaurant, Restaurant",4.3,7752,4.116361
10,Valencia Pizza & Pasta,"Valencia Pizza & Pasta, 801 Valencia St, San F...","Italian restaurant, Breakfast restaurant, Pizz...",4.3,6764,4.102510
8,Monza Pizzeria,"Monza Pizzeria, 1934 Ocean Ave, San Francisco,...",Pizza restaurant,4.6,858,3.976151
1,Tricolore Caffè & Pizzeria and Catering,"Tricolore Caffè & Pizzeria and Catering, 590 W...","Italian restaurant, Bagel shop, Cafe, Catering...",4.5,858,3.964224
5,That's Amore SF,"That's Amore SF, 1901 Ocean Ave, San Francisco...",Pizza restaurant,4.8,175,3.916082


Aquí podemos observar cómo el sistema de puntuación ponderada prevalece sobre el promedio de reseñas: si bien 'Monza Pizzeria' tiene una calificación promedio más alta, tiene una cantidad considerablemente menor de reseñas, por lo que se agrupan de acuerdo a la puntuación ponderada.
Sin embargo, se indica el promedio de calificaciones para que el usuario pueda tenerlo en consideración.

In [53]:
# Probar la función con el DataFrame unificado
recomendaciones_sabor = recomendar_restaurantes_por_sabor_unificado(df_unificado, 'Burger', 'San Diego')
recomendaciones_sabor

,name,address,category,avg_rating,num_of_reviews,puntuacion
8,Crazee Burger,"Crazee Burger, 3993 30th St, San Diego, CA 921...",Hamburger restaurant,4.6,386384,4.515443
7,Veggie Grill,"Veggie Grill, 4353 La Jolla Village Drive #H28...","Vegan restaurant, Gluten-free restaurant, Hamb...",4.5,24208,4.358406
4,Brooklyn Bar & Grill,"Brooklyn Bar & Grill, 4727 University Ave, San...","Bar & grill, American restaurant, Bar, Hamburg...",4.7,3072,4.345842
0,Fatso's Bunker,"Fatso's Bunker, 688 Hollister St ste b, San Di...",Hamburger restaurant,4.8,1170,4.342872
6,The Butcher N Cheese,"The Butcher N Cheese, 4705 Clairemont Dr C, Sa...",Hamburger restaurant,4.6,1020,4.341416


**Mejoramos función para buscar locales por zona**

In [49]:
def recomendar_restaurantes_por_zona(df, ciudad, top_n=5, min_reviews=10):
    '''
    Esta función recibe un DataFrame, y una ciudad o zona del Estado de California.
    Devuelve un top 10 de los mejores locales según la calificación ponderada.
    Pueden no haber reseñas suficientes en la ciudad o zona seleccionada.
    '''
    # Filtrar por ciudad
    df_filtrado = df[df['city'].str.contains(ciudad, case=False, na=False)]
    
    if df_filtrado.empty:
        return f"No se encontraron restaurantes en {ciudad}."
    
    # Filtrar solo restaurantes con un mínimo de reseñas
    df_filtrado = df_filtrado[df_filtrado['num_of_reviews'] >= min_reviews]
    
    if df_filtrado.empty:
        return f"No se encontraron restaurantes con al menos {min_reviews} reseñas en {ciudad}."
    
    # Ordenar por calificación promedio y número de reseñas
    df_top5 = df_filtrado.sort_values(['avg_rating', 'num_of_reviews'], ascending=[False, False]).head(top_n)
    
    # Seleccionar las columnas relevantes para mostrar
    return df_top5[['name', 'avg_rating', 'num_of_reviews', 'category', 'address']]

In [50]:
# Probar la función con el DataFrame df_business
recomendaciones_zona = recomendar_restaurantes_por_zona(df_business, 'San Francisco', top_n=5, min_reviews=20)
recomendaciones_zona

,name,avg_rating,num_of_reviews,category,address
2498158,Serafina,5.0,48,Restaurant,"Serafina, 1701 Jones St, San Francisco, CA 94109"
1956301,Ham and Cheese Deli,5.0,26,Restaurant,"Ham and Cheese Deli, 5501 California St, San F..."
2529977,Cheese Boutique,4.9,68,"Cheese shop, Gourmet grocery store, Sandwich shop","Cheese Boutique, 660 Chenery St, San Francisco..."
955338,Chic n’ Time,4.9,38,"Asian fusion restaurant, Restaurant","Chic n’ Time, 807 Valencia St, San Francisco, ..."
73188,SPRO,4.9,35,"Cafe, Bakery, Coffee shop, Dessert shop, Hambu...","SPRO, 500 Church St, San Francisco, CA 94114"


In [51]:
# Probar la función con el DataFrame df_business
recomendaciones_zona = recomendar_restaurantes_por_zona(df_business, 'Los Angeles', top_n=5, min_reviews=20)
recomendaciones_zona

,name,avg_rating,num_of_reviews,category,address
2504391,Juanito's Tacos,4.9,78,Mexican restaurant,"Juanito's Tacos, 1950 E Washington Blvd, Los A..."
1422509,Eszett,4.9,57,Restaurant,"Eszett, 3510 Sunset Blvd, Los Angeles, CA 90026"
1460247,Cafe De Mama,4.9,38,Coffee shop,"Cafe De Mama, 1102 S Western Ave, Los Angeles,..."
1817065,"50/50 Juice Lounge, Where Health Meets Juice 4...",4.9,38,Restaurant,"50/50 Juice Lounge, Where Health Meets Juice 4..."
2588766,Mo's House Of Axe * Temporarily Closed *,4.9,38,Restaurant,"Mo's House Of Axe * Temporarily Closed *, 611 ..."


In [52]:
# Probar la función con el DataFrame df_business
recomendaciones_zona = recomendar_restaurantes_por_zona(df_business, 'Pasadena', top_n=5, min_reviews=20)
recomendaciones_zona

,name,avg_rating,num_of_reviews,category,address
1672646,Berry Opera,4.9,48,Bakery,"Berry Opera, 811 Fair Oaks Ave, South Pasadena..."
1672648,Bodegón No.69,4.9,25,"Peruvian restaurant, Latin American restaurant...","Bodegón No.69, 69 N Raymond Ave, Pasadena, CA ..."
1589136,Pasadena Victory Park Farmer's Market,4.8,58,"Farmers' market, Food products supplier, Healt...","Pasadena Victory Park Farmer's Market, 2925 E ..."
1093882,HomeState,4.8,38,"Restaurant, Mexican restaurant","HomeState, 1992 Lincoln Ave, Pasadena, CA 91103"
2427333,Teddy's Tacos,4.8,37,Restaurant,"Teddy's Tacos, 1655 E Colorado Blvd, Pasadena,..."


In [54]:
# Probar la función con el DataFrame df_business
recomendaciones_zona = recomendar_restaurantes_por_zona(df_business, 'Anaheim', top_n=5, min_reviews=20)
recomendaciones_zona

,name,avg_rating,num_of_reviews,category,address
1785110,LIT Cafe OC,4.8,58,Brunch restaurant,"LIT Cafe OC, 1071 N Tustin Ave #100, Anaheim, ..."
2477824,Atticus Cafe,4.8,58,Breakfast restaurant,"Atticus Cafe, 1304 S Magnolia Ave, Anaheim, CA..."
152893,Nutrishop,4.8,37,"Vitamin & supplements store, Health food store...","Nutrishop, 30 E Orangethorpe Ave #3, Anaheim, ..."
1935239,Olligo Grill,4.7,137,Restaurant,"Olligo Grill, 3240 Lincoln Ave, Anaheim, CA 92801"
1352824,Ceviche 19,4.7,54,Peruvian restaurant,"Ceviche 19, 1721 W Katella Ave G, Anaheim, CA ..."


**Reformulamos función de búsqueda por palabras clave en reseñas**

In [61]:
def buscar_palabras_clave(df_unificado, palabras_clave, min_reviews=10):
    '''
    Esta función recibe un DataFrame y una lista de palabras clave.
    Busca las reseñas que tengan esas palabras clave, y las ordena según la puntuación ponderada.
    Devuelve una tabla con los locales mejor ponderados que respeten los criterios de búsqueda.
    '''
    # Unir las palabras clave en una expresión regular
    palabras_regex = '|'.join([f'\\b{palabra}\\b' for palabra in palabras_clave])
    
    # Filtrar las reseñas que contienen las palabras clave
    df_filtrado = df_unificado[df_unificado['text'].str.contains(palabras_regex, case=False, na=False)]
    
    if df_filtrado.empty:
        return "No se encontraron reseñas con las palabras clave."
    
    # Agrupar por gmap_id para evitar duplicados
    df_agrupado = df_filtrado.groupby('gmap_id').agg({
        'name': 'first',  # Mantener el primer nombre del restaurante
        'address': 'first',  # Mantener la primera dirección del restaurante
        'city': 'first',  # Mantener la primera ciudad
        'category': 'first',  # Mantener la primera categoría
        'avg_rating': 'mean',  # Promedio de la calificación
        'num_of_reviews': 'sum',  # Sumar el número total de reseñas
        'rating': 'mean',  # Promedio de las calificaciones de las reseñas
        'text': 'first'  # Mantener el primer texto de reseña
    }).reset_index()
    
    # Filtrar solo restaurantes con un mínimo de reseñas
    df_agrupado = df_agrupado[df_agrupado['num_of_reviews'] >= min_reviews]
    
    if df_agrupado.empty:
        return f"No se encontraron restaurantes con al menos {min_reviews} reseñas que contengan las palabras clave."
    
    # Ordenar por calificación promedio y número de reseñas
    df_top = df_agrupado.sort_values(['avg_rating', 'num_of_reviews'], ascending=[False, False])
    
    # Seleccionar las columnas relevantes para mostrar
    return df_top[['name', 'address', 'city', 'category', 'avg_rating', 'num_of_reviews', 'text']].head(10)



In [62]:
# Probar la función con df_unificado
palabras_clave = ['amazing', 'good', 'very good', 'friendly']
resultados_busqueda = buscar_palabras_clave(df_unificado, palabras_clave)
resultados_busqueda

,name,address,city,category,avg_rating,num_of_reviews,text
5029,The Kilted Cake and Supply,"The Kilted Cake and Supply, 28061 Jefferson Av...",Temecula,"Baking supply store, Airbrushing supply store,...",5.0,240,So nice! Extremely organized. Amazing customer...
267,LASO /Nepalese Fusion,"LASO /Nepalese Fusion, 101 Antonina Ave Suite ...",American Canyon,Fusion restaurant,5.0,225,Wife and I had an excellent dinner at the outd...
5391,In-N-Out Burger,"In-N-Out Burger, 3483 Van Buren Boulevard, Riv...",Riverside,"Hamburger restaurant, American restaurant, Din...",5.0,146,Very good service dine in or dine out its a gr...
2568,North Fork Chai Co.,"North Fork Chai Co., 661 Suite B, Newcastle Rd...",Newcastle,Cafe,5.0,138,Great place to come to for some tra or coffee....
2518,FUKUMI RAMEN - Citrus Heights,"FUKUMI RAMEN - Citrus Heights, 5410 Sunrise Bl...",Citrus Heights,Japanese restaurant,5.0,100,So nice to have good ramen in the area! The po...
4947,Gabino's Creperie,"Gabino's Creperie, 170 E Palm Canyon Dr #4, Pa...",Palm Springs,Restaurant,5.0,96,I would start with saying this place is a hidd...
4715,San Diego Kabob Shack,"San Diego Kabob Shack, 200 Broadway, Chula Vis...",Chula Vista,Restaurant,5.0,94,We heard of San Diego Kabob Shack from the Eas...
5175,CALVIN'S KOREAN HOT CHICKEN,"CALVIN'S KOREAN HOT CHICKEN, 3211 Holiday Ct S...",La Jolla,"Fried chicken takeaway, Chicken shop, Chicken ...",5.0,92,Wow. So very good. Also they have gluten free ...
6346,Reverent Coffee Bar,"Reverent Coffee Bar, 627 Deep Valley Dr Suite ...",Rolling Hills Estates,Coffee shop,5.0,78,I've been here only a few times so far - which...
4711,BAJA FISH TACOS & BURGERS,"BAJA FISH TACOS & BURGERS, 71 Broadway, Chula ...",Chula Vista,"Seafood restaurant, Lunch restaurant",5.0,76,I got the fish burrito and the ceviche.. don't...


In [63]:
# Probar la función con df_unificado
palabras_clave = ['good burger']
resultados_busqueda = buscar_palabras_clave(df_unificado, palabras_clave)
resultados_busqueda

,name,address,city,category,avg_rating,num_of_reviews,text
44,Umai Savory Hot Dogs,"Umai Savory Hot Dogs, 8443 Haven Ave #183, Ran...",Rancho Cucamonga,"Hot dog restaurant, American restaurant, Chick...",4.8,108,My bf and I love a good burger. Umai has a pre...
20,In-N-Out Burger,"In-N-Out Burger, 8010 N Blackstone Ave, Fresno...",Fresno,"Hamburger restaurant, American restaurant, Din...",4.7,2408,Aways count on Good Burger and Great fries.
32,In-N-Out Burger,"In-N-Out Burger, 11880 Carmel Mountain Rd, Car...",Carmel Mountain Ranch,"Hamburger restaurant, American restaurant, Din...",4.7,1685,good burger
10,Hidden Spot,"Hidden Spot, 303 Grand Ave, South San Francisc...",South San Francisco,Hamburger restaurant,4.7,67,Just a really good burger.. I had the classic.
1,In-N-Out Burger,"In-N-Out Burger, 1275 Dana Dr, Redding, CA 960...",Redding,"Hamburger restaurant, American restaurant, Din...",4.6,70952,Good place to go for a good burger and fries a...
19,In-N-Out Burger,"In-N-Out Burger, 4302 N Blackstone Ave, Fresno...",Fresno,"Hamburger restaurant, American restaurant, Din...",4.6,14844,Good burger. Long wait in drive thru
24,In-N-Out Burger,"In-N-Out Burger, 1933 S Mooney Blvd, Visalia, ...",Visalia,"Hamburger restaurant, American restaurant, Din...",4.6,12234,AWESOME FOOD & EXPEDIENCY AFTER ORDERING. YOU ...
71,In-N-Out Burger,"In-N-Out Burger, 600 S Brookhurst St, Anaheim,...",Anaheim,"Hamburger restaurant, American restaurant, Din...",4.6,7238,Great place for quick bites.Not to expect anyt...
18,In-N-Out Burger,"In-N-Out Burger, 3900 Pelandale Ave, Modesto, ...",Modesto,"Hamburger restaurant, American restaurant, Din...",4.6,5337,Good burger and a very pleasant staff
66,Burger Lounge,"Burger Lounge, 922 Orange Ave, Coronado, CA 92...",Coronado,"Hamburger restaurant, American restaurant",4.6,4880,"Good burger, you can tell in the taste that ev..."


In [64]:
# Probar la función con df_unificado
palabras_clave = ['excelent service']
resultados_busqueda = buscar_palabras_clave(df_unificado, palabras_clave)
resultados_busqueda

,name,address,city,category,avg_rating,num_of_reviews,text
7,Deliciosas Pupusas inside car wash,"Deliciosas Pupusas inside car wash, 5509 Alham...",Los Angeles,Restaurant,4.8,18,Excelent service and delicious food!!
6,Saigon Dish Vietnamese Restaurant,"Saigon Dish Vietnamese Restaurant, Spires Cour...",Lawndale,"Vietnamese restaurant, Asian restaurant, Resta...",4.6,1088,Best place for vietnamese soup. Excelent service.
9,La Bella Pizza,"La Bella Pizza, 373 3rd Ave, Chula Vista, CA 9...",Chula Vista,"Italian restaurant, Banquet hall, Caterer, Tak...",4.4,2248,(Translated by Google) Excelent service\n\n(Or...
5,Las Lomas Restaurant,"Las Lomas Restaurant, 15677 Roxford St, Sylmar...",Sylmar,Mexican restaurant,4.4,438,(Translated by Google) Excelent service\n\n(Or...
1,La Buena Taqueria and More,"La Buena Taqueria and More, 1425 Main St, Wats...",Watsonville,"Fast food restaurant, Hamburger restaurant, Ho...",4.4,108,(Translated by Google) Excelent service\n\n(Or...
13,Sabor A Malibu,"Sabor A Malibu, 31242 CA-1, Malibu, CA 90265",Malibu,"Taco restaurant, Restaurant",4.4,34,(Translated by Google) Excelent service\n\n(Or...
14,Cool Hand Luke's - Santa Maria,"Cool Hand Luke's - Santa Maria, 1321 s, Nichol...",Santa Maria,"Steak house, American restaurant",4.3,876,Excelent service
8,Tom's Jr.,"Tom's Jr., 9856 Atlantic Ave, South Gate, CA 9...",South Gate,"Hamburger restaurant, Fast food restaurant",4.2,1324,(Translated by Google) Excelent service\n\n(Or...
12,Denny's,"Denny's, 1440 W Pacific Coast Hwy, Wilmington,...",Wilmington,"Diner, American restaurant, Breakfast restaurant",3.9,1497,(Translated by Google) Excelent service\n\n(Or...
10,Jack in the Box,"Jack in the Box, 1900 Mission Ave, Oceanside, ...",Oceanside,"Fast food restaurant, Breakfast restaurant, Ch...",3.9,721,(Translated by Google) Excelent service\n\n(Or...


In [65]:
# Probar la función con df_unificado
palabras_clave = ['fresh seafood']
resultados_busqueda = buscar_palabras_clave(df_unificado, palabras_clave)
resultados_busqueda

,name,address,city,category,avg_rating,num_of_reviews,text
6,O' by Claude Le Tohic,"O' by Claude Le Tohic, 165 O'Farrell St 5th Fl...",San Francisco,"Fine dining restaurant, French restaurant, Res...",4.9,25,Swung in for dinner and my first concern was t...
30,Poki Roll,"Poki Roll, 5721 Lincoln Ave c, Cypress, CA 906...",Cypress,Sushi restaurant,4.8,266,Nice and fresh seafood!
0,Princess Seafood Market & Deli,"Princess Seafood Market & Deli, 32410 N Harbor...",Fort Bragg,Seafood restaurant,4.7,5180,Delicious fresh seafood in a bright and clean ...
18,Ensenada's Surf N Turf Grill,"Ensenada's Surf N Turf Grill, 4749 Artesia Blv...",Lawndale,"Seafood restaurant, Restaurant",4.7,2222,"Fresh seafood, great customer service, very fr..."
10,Evangeline Cuisine,"Evangeline Cuisine, 225 Cabrillo Hwy S #102 C,...",Half Moon Bay,"Seafood restaurant, Coffee shop",4.7,112,"Amazing paella, one of the best I've had, and ..."
2,Poke Go,"Poke Go, 3555 Mt Diablo Blvd, Lafayette, CA 94549",Lafayette,Restaurant,4.7,68,Great fresh seafood
34,Kikos Seafood Lunch Truck,"Kikos Seafood Lunch Truck, 6090 Friars Rd, San...",San Diego,Seafood restaurant,4.7,54,I mean what can I say.....Everything is great ...
29,Original Fish Co.,"Original Fish Co., 11061 Los Alamitos Blvd, Lo...",Los Alamitos,Restaurant,4.6,7518,Been coming here for about 30 years. While I'...
25,California Fish Grill,"California Fish Grill, 1530 Camino De La Reina...",San Diego,Seafood restaurant,4.6,4232,Lot of choice for fresh seafood for reasonable...
35,Lure Fish House,"Lure Fish House, 3815 State St Suite G131, San...",Santa Barbara,"Seafood restaurant, Restaurant",4.6,1088,This place is hit or miss on service. I’ve had...


**Reformulamos la función de recomendaciones en base a reseñas**

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Esta función no se utiliza, ya que tenía algunos problemas:

In [ ]:

def recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante, top_n=5):
    # Filtrar el DataFrame para encontrar el restaurante
    df_restaurante = df_unificado[df_unificado['name'].str.contains(nombre_restaurante, case=False, na=False)]
    
    if df_restaurante.empty:
        return f"No se encontraron restaurantes con el nombre {nombre_restaurante}."
    
    # Obtener el gmap_id del primer restaurante encontrado (asumiendo que es único)
    gmap_id = df_restaurante.iloc[0]['gmap_id']
    
    # Filtrar las reseñas que tienen texto y corresponden a este restaurante
    df_filtrado = df_unificado[(df_unificado['text'].notnull()) & (df_unificado['gmap_id'] == gmap_id)]
    
    if df_filtrado.empty:
        return f"No se encontraron reseñas para el restaurante {nombre_restaurante}."
    
    # Resetear el índice del DataFrame filtrado para evitar problemas con el índice
    df_filtrado = df_filtrado.reset_index(drop=True)
    
    # Crear la matriz TF-IDF
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_filtrado['text'])
    
    # Calcular la similitud del coseno entre todas las reseñas
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Obtener el índice del restaurante en la matriz
    idx = 0  # Siempre usamos el primer índice ya que reiniciamos el DataFrame
    
    # Verificar que la matriz de similitud tiene el tamaño correcto
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar por similitud y devolver los más similares
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n + 1]
    
    # Obtener los índices de las reseñas recomendadas
    review_indices = [i[0] for i in sim_scores]
    
    # Obtener información del restaurante para las reseñas recomendadas
    df_recomendaciones = df_unificado[df_unificado['gmap_id'] == gmap_id].drop_duplicates(subset='gmap_id')
    
    # Devolver la información de los restaurantes recomendados junto con las reseñas
    recomendaciones = df_recomendaciones[['name', 'address', 'city', 'category', 'avg_rating', 'num_of_reviews']].reset_index(drop=True)
    
    # Asegurarse de que la longitud de reseñas coincida con la de recomendaciones
    if len(recomendaciones) > 0:
        # Reasignar las reseñas si hay al menos una recomendación
        recomendaciones['reseñas'] = [''] * len(recomendaciones)
        for idx, review_index in enumerate(review_indices):
            if idx < len(recomendaciones):
                recomendaciones.at[idx, 'reseñas'] = df_filtrado.iloc[review_index]['text']
    
    return recomendaciones



In [70]:
# Probar la función con df_unificado
nombre_restaurante = 'Pizza Hut'
resultados_recomendaciones = recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante)
resultados_recomendaciones

,name,address,city,category,avg_rating,num_of_reviews,reseñas
0,Pizza Hut,"Pizza Hut, 7750 El Camino Real suite e, Carlsb...",Carlsbad,"Pizza restaurant, Chicken wings restaurant, De...",3.6,28,Good


In [71]:
# Probar la función con df_unificado
nombre_restaurante = 'Starbucks'
resultados_recomendaciones = recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante)
resultados_recomendaciones

,name,address,city,category,avg_rating,num_of_reviews,reseñas
0,Starbucks,"Starbucks, Newport FS Town Center, Menifee, CA...",Menifee,Coffee shop,4.5,26,Finally a drive thru that's close to the house...


In [73]:
# Probar la función con df_unificado
nombre_restaurante = 'Burger King'
resultados_recomendaciones = recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante)
resultados_recomendaciones

,name,address,city,category,avg_rating,num_of_reviews,reseñas
0,Burger King,"Burger King, 3030 Plaza Bonita Rd Suite 2075, ...",National City,Fast food restaurant,2.5,15,


Con el enfoque que se le dió a esta función, se buscan reseñas relacionadas con un restaurante y luego utiliza esas reseñas para recomendar el mismo restaurante. De esa forma, decidimos cambiar el enfoque para que efectivamente esta recomendación sea de utilidad.

In [77]:
# Probar la función con df_unificado
nombre_restaurante = 'Pizza Hut'
resultados_recomendaciones = recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante)
resultados_recomendaciones

,name,address,city,category,avg_rating,num_of_reviews,text
0,JiST cafe,"JiST cafe, 116 Judge John Aiso St, Los Angeles...",Los Angeles,"Cafe, New American restaurant",4.4,448,Don't be a hater is place is great! I can't st...
1,Yashima Restaurant,"Yashima Restaurant, 236 E Rowland St, Covina, ...",Covina,"Japanese restaurant, Restaurant",4.7,178,WHAT IT IS\nSushi restaurant and bar. They als...
2,SUSHI YAN,"SUSHI YAN, 9047 Central Ave, Montclair, CA 91763",Montclair,Sushi restaurant,4.4,56,So I saw the reviews and decided to give this ...
3,In-N-Out Burger,"In-N-Out Burger, 600 S Brookhurst St, Anaheim,...",Anaheim,"Hamburger restaurant, American restaurant, Din...",4.6,3619,"I mean, in n out, is in n out. The burgers are..."
4,La Ranchera Mexican Food,"La Ranchera Mexican Food, 911 E La Habra Blvd,...",La Habra,"Mexican restaurant, Latin American restaurant",4.1,177,My review is based on delivery. I’ve never bee...


In [78]:
# Probar la función con df_unificado
nombre_restaurante = 'Starbucks'
resultados_recomendaciones = recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante)
resultados_recomendaciones

,name,address,city,category,avg_rating,num_of_reviews,text
0,Starbucks,"Starbucks, Newport FS Town Center, Menifee, CA...",Menifee,Coffee shop,4.5,26,I love the new location! They were originally...
1,Starbucks,"Starbucks, 11697 Del Amo Blvd, Lakewood, CA 90...",Lakewood,"Coffee shop, Breakfast restaurant, Cafe, Coffe...",4.2,285,Q) Did you attend the Grand Opening of this S...
2,Pizza Man,"Pizza Man, 10940 W Magnolia Blvd, North Hollyw...",North Hollywood,"Pizza restaurant, Pizza delivery",3.9,167,2019.01-03: Can't wait for my order! It's been...
3,Friends With Benedicts Mimosa House,"Friends With Benedicts Mimosa House, Palladio ...",Folsom,Breakfast restaurant,3.9,18,Found this new location on our way out from sh...


En las primeras pruebas, observamos que si bien se sigue un patrón de similitud para la recomendación, se aleja demasiado de la categoría. Hay más coincidencia, por ejemplo, cuando buscamos cafeterías. Para mejorar esta cuestión, debemos contemplar la categoría como otro factor. Agregaremos una columna combinada que tenga las palabras de la columna 'category' (varias veces para darle más preponderancia) y 'text', y la búsqueda de coincidencias se haga a partir de esa nueva columna.

In [91]:
# Crear una columna combinada 'category_text' con la categoría repetida 3 veces para darle mayor peso
df_unificado['category_text'] = df_unificado['category'] + ' ' + df_unificado['category'] + ' ' + df_unificado['category'] + ' ' + df_unificado['text']


## Sistema final de recomendación en base a reseñas

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante, top_n=5):
    # Filtrar el DataFrame para encontrar el restaurante
    df_restaurante = df_unificado[df_unificado['name'].str.contains(nombre_restaurante, case=False, na=False)]
    
    if df_restaurante.empty:
        return f"No se encontraron restaurantes con el nombre {nombre_restaurante}."
    
    # Obtener el gmap_id del restaurante
    gmap_id_restaurante = df_restaurante.iloc[0]['gmap_id']
    
    # Filtrar las reseñas que tienen texto y corresponden a este restaurante
    df_reseñas_restaurante = df_unificado[(df_unificado['text'].notnull()) & (df_unificado['gmap_id'] == gmap_id_restaurante)]
    
    if df_reseñas_restaurante.empty:
        return f"No se encontraron reseñas para el restaurante {nombre_restaurante}."
    
    # Crear la matriz TF-IDF para las reseñas combinadas
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix_reseñas = tfidf.fit_transform(df_reseñas_restaurante['category_text'])
    tfidf_matrix_todas_reseñas = tfidf.transform(df_unificado['category_text'])
    
    # Calcular la similitud del coseno entre las reseñas del restaurante y todas las reseñas
    cosine_sim = cosine_similarity(tfidf_matrix_reseñas, tfidf_matrix_todas_reseñas)
    
    # Obtener el índice de reseñas más similares
    sim_scores = list(enumerate(cosine_sim.mean(axis=0)))  # Usar la media de similitud para comparar con todas las reseñas
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:top_n]
    
    # Obtener los índices de los restaurantes recomendados
    restaurant_indices = [i[0] for i in sim_scores]
    
    # Asegurarse de que los índices están dentro del rango del DataFrame
    if restaurant_indices:
        # Obtener información de los restaurantes recomendados
        recomendaciones = df_unificado.iloc[restaurant_indices].drop_duplicates(subset='gmap_id')
        recomendaciones = recomendaciones[['name', 'address', 'city', 'category', 'avg_rating', 'num_of_reviews', 'text']].reset_index(drop=True)
    else:
        recomendaciones = pd.DataFrame()  # Devolver un DataFrame vacío si no hay recomendaciones
    
    return recomendaciones


In [94]:
nombre_restaurante = 'Vons Chicken'
resultados_recomendaciones = recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante)
resultados_recomendaciones


,name,address,city,category,avg_rating,num_of_reviews,text
0,Crab House at Pier 39,"Crab House at Pier 39, San Francisco, 203 Pier...",San Francisco,"Crab house, American restaurant, Bar, Californ...",4.2,2068,(Translated by Google) It has a different expe...
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",La Mirada,Restaurant,4.5,18,(Translated by Google) There is a different me...
2,Hong Kong Flower Lounge Restaurant 香滿樓,"Hong Kong Flower Lounge Restaurant 香滿樓, 51 E M...",Millbrae,"Chinese restaurant, Dim sum restaurant, Seafoo...",3.9,918,(Translated by Google) The taste is good.\n\nH...
3,Tom's Jr,"Tom's Jr, 1725 N Long Beach Blvd, Compton, CA ...",Compton,"Hamburger restaurant, Fast food restaurant, Re...",4.4,1248,(Translated by Google) I love buying the famil...
4,Olympian Burgers,"Olympian Burgers, 2701 S Vermont Ave, Los Ange...",Los Angeles,"Family restaurant, American restaurant, Fast f...",4.5,1500,(Translated by Google) A place with many years...


In [95]:
nombre_restaurante = 'Pizza Hut'
resultados_recomendaciones = recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante)
resultados_recomendaciones


,name,address,city,category,avg_rating,num_of_reviews,text
0,Pizza Hut,"Pizza Hut, 7750 El Camino Real suite e, Carlsb...",Carlsbad,"Pizza restaurant, Chicken wings restaurant, De...",3.6,28,Good
1,Pizza Hut,"Pizza Hut, 8947 W Pico Blvd, Los Angeles, CA 9...",Los Angeles,"Pizza restaurant, Chicken wings restaurant, De...",3.0,48,It's okay
2,Pizza Hut,"Pizza Hut, 415 E Avenida Pico Ste K, San Cleme...",San Clemente,"Pizza restaurant, Chicken wings restaurant, De...",2.9,38,I love this place...greatest ever
3,Pizza Hut,"Pizza Hut, 92 W Court St, Woodland, CA 95695",Woodland,"Pizza restaurant, Chicken wings restaurant, De...",3.7,38,I loved it😁


In [96]:
nombre_restaurante = 'Starbucks'
resultados_recomendaciones = recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante)
resultados_recomendaciones


,name,address,city,category,avg_rating,num_of_reviews,text
0,Starbucks,"Starbucks, Newport FS Town Center, Menifee, CA...",Menifee,Coffee shop,4.5,26,Finally a drive thru that's close to the house...
1,Starbucks,"Starbucks, 11697 Del Amo Blvd, Lakewood, CA 90...",Lakewood,"Coffee shop, Breakfast restaurant, Cafe, Coffe...",4.2,285,Q) Did you attend the Grand Opening of this S...
2,Starbucks,"Starbucks, 78752 CA-111, La Quinta, CA 92253",La Quinta,"Coffee shop, Cafe",3.9,47,New location in Palm Desert. Good location
3,Starbucks,"Starbucks, 4110 Norwood Ave, Sacramento, CA 95...",Sacramento,"Coffee shop, Breakfast restaurant, Cafe, Coffe...",4.0,955,Some staff are wonderful. Location is main dra...


In [97]:
nombre_restaurante = 'Subway'
resultados_recomendaciones = recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante)
resultados_recomendaciones


,name,address,city,category,avg_rating,num_of_reviews,text
0,Subway,"Subway, 124 Lomas Santa Fe Dr Suite 101, Solan...",Solana Beach,"Sandwich shop, Caterer, Fast food restaurant, ...",3.5,58,Poor customer service. First the employee trie...
1,Subway,"Subway, 2138 Hilltop Mall Rd Kiosk K127, Richm...",Richmond,"Sandwich shop, Fast food restaurant, Takeout R...",3.7,49,This location is good! The Subs were once alwa...
2,Subway,"Subway, 11123 Long Beach Blvd Unit 10, Lynwood...",Lynwood,"Sandwich shop, Caterer, Fast food restaurant, ...",3.8,103,I liked this location.
3,Subway,"Subway, 83097 Avenue 48 # D1a, Coachella, CA 9...",Coachella,"Sandwich shop, Caterer, Fast food restaurant, ...",3.5,58,Love this location


In [98]:
nombre_restaurante = '21 Biryani Pots'
resultados_recomendaciones = recomendar_restaurantes_por_reseñas(df_unificado, nombre_restaurante)
resultados_recomendaciones

,name,address,city,category,avg_rating,num_of_reviews,text
0,Paradise Biryani Pointe,"Paradise Biryani Pointe, 8995 Mira Mesa Blvd S...",San Diego,Indian restaurant,3.5,618,Authentic biryani place. Their biryani is very...
1,Sakoon,"Sakoon, 357 Castro St, Mountain View, CA 94041...",Mountain View,"Indian restaurant, Bar",4.2,2474,Very Authentic Indian food.\nHappy Hour for dr...
2,Biryani House,"Biryani House, 949 Ruff Dr, San Jose, CA 95110",San Jose,"Indian takeaway, Biryani restaurant, Indian re...",4.7,68,Incredible place that deserves more ratings an...


Este sistema funciona mejor con pequeños negocios que con grandes cadenas, porque cada sucursal de las cadenas (al tratarse de franquicias), comparten reseñas similares. Como puede verse, un pequeño local de comida India me trae restaurantes o locales similares del Estado de California.